In [ ]:
import plotly.express as px

df = px.data.gapminder()
fig = px.choropleth(df, locations="iso_alpha", color="lifeExp", hover_name="country", animation_frame="year", range_color=[20,80])
fig.show()


In [ ]:
import plotly.express as px

df = px.data.election()
geojson = px.data.election_geojson()

fig = px.choropleth_mapbox(df, geojson=geojson, color="Bergeron",
                           locations="district", featureidkey="properties.district",
                           center={"lat": 45.5517, "lon": -73.7073},
                           mapbox_style="carto-positron", zoom=9)
fig.show()

In [ ]:
import plotly.express as px
df = px.data.gapminder()
fig = px.scatter_geo(df, locations="iso_alpha", color="continent", hover_name="country", size="pop",
               animation_frame="year", projection="natural earth")
fig.show()

In [ ]:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_mapbox(df, lat="centroid_lat", lon="centroid_lon", color="peak_hour", size="car_hours",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10,
                  mapbox_style="carto-positron")
fig.show()

In [ ]:
import plotly.express as px

df = px.data.gapminder().query("year == 2007")
fig = px.sunburst(df, path=['continent', 'country'], values='pop',
                  color='lifeExp', hover_data=['iso_alpha'])
fig.show()

In [ ]:
import plotly.express as px
df = px.data.gapminder().query("year == 2007").query("continent == 'Europe'")
df.loc[df['pop'] < 2.e6, 'country'] = 'Other countries' # Represent only large countries
fig = px.pie(df, values='pop', names='country', title='Population of European continent')
fig.show()

In [ ]:
def mcrores(number):
    return '₹'+'{:,.0f} Cr'.format(round(number / 10000000))
def lakh(number):
    return '{:.2f}L'.format(number / 100000)

def formated(number):
    number_str = str(number)
    length = len(number_str)
    formatted_number = ""
    comma_counter = 0
    for i in range(length - 1, -1, -1):
        formatted_number = number_str[i] + formatted_number
        comma_counter += 1
        if comma_counter == 2 and i != 0:
            formatted_number = "," + formatted_number
            comma_counter = 0
        elif comma_counter == 3 and i != 0:
            formatted_number = "," + formatted_number
            comma_counter = 0
    return formatted_number



In [ ]:
import pandas as pd
import mysql.connector as sql
import requests
import json
import plotly_express as px


db = sql.connect(host="localhost",user="root",password="kobalan",auth_plugin="mysql_native_password",database="phonepe")

cursor = db.cursor()
year=2022
quarter=2
cursor.execute(f'select * from agg_transaction')
df_Trans1=pd.DataFrame(cursor.fetchall(),columns=cursor.column_names)
df_Trans2=df_Trans1[(df_Trans1['Year']==year) & (df_Trans1['Quarter']==quarter)]
df_Trans2= df_Trans2.groupby("State")[["Transaction_count", "Transaction_amount"]].sum().sort_values(by='Transaction_count',ascending=False)
df_Trans2.reset_index(inplace= True)
df_Trans2['Avg.Transaction Value']=df_Trans2['Transaction_amount']//df_Trans2['Transaction_count']
df_Trans2['State'] = df_Trans2['State'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")
df_Trans2['State'] = df_Trans2['State'].str.replace("Andaman & Nicobar ","Andaman & Nicobar")
df_Trans2['Avg_Transaction_Value'] = df_Trans2['Avg.Transaction Value'].apply(lambda x: round(x)).apply(lambda x: "₹{:,.0f}".format(x))
df_Trans2['All Transaction'] = df_Trans2['Transaction_count'].apply(lambda x: round(x)).apply(lambda x: formated(x))
df_Trans2['Total Payment Value'] = df_Trans2['Transaction_amount'].apply(lambda x: round(x)).apply(lambda x: mcrores(x))


url= "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
response= requests.get(url)
data= json.loads(response.content)
states_name= [feature["properties"]["ST_NM"] for feature in data["features"]]




fig= px.choropleth(df_Trans2, geojson= data, locations= "State", featureidkey= "properties.ST_NM",
                            color= "Transaction_amount", color_continuous_scale= "ylorrd",
                            range_color= (df_Trans2["Transaction_amount"].min(),df_Trans2["Transaction_amount"].max()),
                            hover_name= "State",title ='Map_View',
                            # hover_data=['Total Payment Value','All Transaction','Avg_Transaction_Value'],
                            hover_data={'All Transaction':True,'Total Payment Value':True,'State':False,'Avg_Transaction_Value':True,'Transaction_amount':False},
                            fitbounds= "locations",width =1000, height= 600)

fig.update_geos(visible =False)
fig.update_layout(margin={"r":80,"t":80,"l":80,"b":80})

In [ ]:
def count_Convert (number):
        if len(number)>=8:
                return '{:,.2f} Cr'.format(int(number) / 10000000)
        elif len(number)==7 or len(number)==6:
                return '{:.2f}L'.format(int(number) / 100000)
        
#turbo,Viridis,ylorrd

In [ ]:
import plotly.graph_objects as go

with open('india_state_geo.json') as response:
    geo=json.load(response)

fig = go.Figure(
    go.Choroplethmapbox(
        geojson=geo,
        locations=df_Trans2.State,
        featureidkey="properties.ST_NM",
        z=df_Trans2['Transaction_count'],
        colorscale="viridis",
        # zmin=0,
        # zmax=500000,
        marker_opacity=0.5,
        marker_line_width=0,
    )
)
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=4.0,
    mapbox_center={"lat": 24, "lon": 79},
    width=800,
    height=800,
)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
import requests
import json

url= "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
response= requests.get(url)
data= json.loads(response.content)
states_name= [feature["properties"]["ST_NM"] for feature in data["features"]]
# State1=sorted(states_name)
# State2=df_Trans2['State'].sort_values(ascending=True).tolist()
# for i in range(len(states_name)):
#     if State1[i]==State2[i]:
#         print("YES")
#     else:
#         print("NO")
# print(states_name[0])

# print(State1)
# print(State2)
# print(df_Trans2['id'])
states_name

In [ ]:
cursor.execute(f'select * from agg_insurance')
df_Trans1=pd.DataFrame(cursor.fetchall(),columns=cursor.column_names)
df_Trans2=df_Trans1[(df_Trans1['Year']==int(year)) & (df_Trans1['Quarter']==int(quarter))]

df_Trans2= df_Trans2.groupby("State")[["Transaction_count", "Transaction_amount"]].sum().sort_values(by='Transaction_count',ascending=False)
df_Trans2.reset_index(inplace= True)
df_Trans2['Avg.Transaction Value']=df_Trans2['Transaction_amount']//df_Trans2['Transaction_count']
df_Trans2['State'] = df_Trans2['State'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Daman & Diu")

# df_Trans2['Avg Premium Value'] = df_Trans2['Avg.Transaction Value'].apply(lambda x: round(x)).apply(lambda x: "₹{:,.0f}".format(x))
# df_Trans2['Insurance Policies Nos.'] = df_Trans2['Transaction_count'].apply(lambda x: round(x)).apply(lambda x: comma_Seperator(x))
# df_Trans2['Total Premium Value'] = df_Trans2['Transaction_amount'].apply(lambda x: round(x)).apply(lambda x: total_Count(str(x)))


#Transaction_Map................

url= "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
response= requests.get(url)
data= json.loads(response.content)
states_name= [feature["properties"]["ST_NM"] for feature in data["features"]]
print(sorted(df_Trans2['State']))


In [ ]:
url= "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
response= requests.get(url)
regions=[]
data= json.loads(response.content)
states_name= [feature["properties"]["ST_NM"] for feature in data["features"]]
regions.append(states_name)
fig1 = px.choropleth_mapbox(
    df_Trans2,
    locations=df_Trans2.State,
    geojson=geo,
    hover_name="State",
    color="Transaction_amount",
   #hover_data={'All Transaction':True,'Total Payment Value':True,'Avg.Transaction Value':True,'Transaction_amount':False},
    title=f"PhonePe Amounts Transactions in Q {quarter}-{year}",
    mapbox_style="carto-positron",
    center={"lat": 24, "lon": 79},
    color_continuous_scale=px.colors.diverging.PuOr,
    color_continuous_midpoint=0,
    zoom=3.6,
    width=800, 
    height=800
)
fig1.update_layout(coloraxis_colorbar=dict(title=' ', showticklabels=True),title={
        'font': {'size': 24}
    },hoverlabel_font={'size': 18})

In [ ]:
year=2018
quarter=1
# state=df.Trans2.State.tolist()
state="Karnataka"
cursor.execute(f'select State,Pincode,sum(Transaction_amount) as Total_Payment_value,sum(Transaction_count) as Transactions  from top_transaction where year={year} and quarter={quarter} group by Pincode,State order by Total_Payment_value desc')
df_Trans3=pd.DataFrame(cursor.fetchall(),columns=cursor.column_names)

df_Trans4=df_Trans3[df_Trans3['State']==state]

res_df=df_Trans4.Pincode.tolist()
res_df=[str(i) for i in res_df]
print(res_df)
print(type(res_df[0]))
# df_Trans4['Avg_Transaction_Value'] = df_Trans4['Avg.Transaction Value'].apply(lambda x: round(x)).apply(lambda x: "₹{:,.0f}".format(x))
# df_Trans4['All Transaction'] = df_Trans4['Transaction_count'].apply(lambda x: round(x)).apply(lambda x: formated(x))
# df_Trans4['Total Payment Value'] = df_Trans4['Transaction_amount'].apply(lambda x: round(x)).apply(lambda x: mcrores(x))

In [ ]:
result="Tamil Nadu"

cursor.execute(f'select State,Pincode,sum(Transaction_amount) as Total_Payment_value,sum(Transaction_count) as Total_Transaction  from top_transaction where year={year} and quarter={quarter} group by Pincode,State order by Total_Payment_value desc')
df_Trans5=pd.DataFrame(cursor.fetchall(),columns=cursor.column_names)
df_Trans6=df_Trans5[df_Trans5['State']==result]
pincode1=df_Trans6.Pincode.tolist()
pincode2=[str(i) for i in pincode1]
count3=df_Trans6.Total_Transaction.tolist()
fig3=go.Figure(go.Bar(
            x=pincode2,                                                           # Labels for the x-axis
            y=count3,                                                   # Values for the y-axis
            name="Pincode",                                         # Name for the trace (used in legends)
            marker=dict(color='red'),   
            orientation="v",
                                        
))
# fig3.update_layout(barmode="stack")
fig3.update_layout(
    width=600,
    height=600,
    title=f"PhonePe Amounts Transactions in Q-{quarter}-{year} Pincode Wise",
)
fig3.show()

In [ ]:
def crores(number):
    return '{:.2f}Cr'.format(number / 10000000)

def formated(number):
    number_str = str(number)
    length = len(number_str)
    formatted_number = ""
    comma_counter = 0
    for i in range(length - 1, -1, -1):
        formatted_number = number_str[i] + formatted_number
        comma_counter += 1
        if comma_counter == 2 and i != 0:
            formatted_number = "," + formatted_number
            comma_counter = 0
        elif comma_counter == 3 and i != 0:
            formatted_number = "," + formatted_number
            comma_counter = 0
    return formatted_number

## Users section values
def format_number1(number):
    number_str = "{:,.0f}".format(number)
    return number_str.replace(",", ",")
def lakh(number):
    return '{:.2f}L'.format(number / 100000)

In [ ]:
filter_tr = tr.loc[(tr['Year']==int(year)) & (tr['Quarter']==int(quarter))]
gr_tr = filter_tr.groupby('Year').sum()
All_transactions = gr_tr['TotalTransactionCount'].to_list()[0]
Total_payments =gr_tr['TotalTransactionAmount'] #for formating
Total_payments1 =gr_tr['TotalTransactionAmount'].to_list()[0]# ****All Transaction****
reversed_numbers = [segment[:] for segment in str(All_transactions).split(",")]
reversed_number = ",".join(reversed_numbers)
def format_number(number):
    return "{:,}".format(number)
atl = format_number(All_transactions)
#atl = "{:,}".format(All_transactions)
Avg_Transaction = round(Total_payments1/All_transactions)# *** Averege transaction value
av_form = '₹{:,}'.format(Avg_Transaction)
# Set the locale to Indian English
sf1 = Total_payments.apply(lambda x: "₹" + "{:,.0f}".format(x/10000000) + "Cr")

trvalue1 = sf1.to_list()[0] # ***Total payments 

In [ ]:
num = 1000000
print(f"{num:,}")


In [ ]:

import babel.numbers
num1=1000000

def comma_Seperator(num):
    result=babel.numbers.format_currency(num,'INR',locale="en_IN")
    v=len(result)-3
    formatted_Value=result[1:v]
    return formatted_Value



In [ ]:
comma_Seperator(1000)

In [ ]:
round_Value=lambda x: round(x)
X=1764000/100000
print(round_Value(X))

In [ ]:
cursor.execute(f'select State,Pincode,sum(Registered_users) as Total_Registered_Users  from top_Users where year=2018 and quarter=3 group by Pincode,State order by Total_Registered_Users desc')
#cursor.execute(f'select State,Pincode,sum(Registered_users) as Total_R from top_Users where year=2018 and quarter=2 group by pincode,State order by Total_R desc')
df_Trans5=pd.DataFrame(cursor.fetchall(),columns=cursor.column_names)
print(df_Trans5.head())
#State,Pincode,sum(Registered_users) as Total Registered Users  from top_Users where year={year} and quarter={quarter} group by Pincode,State order by Total Registered Users desc

In [ ]:
x=0.9999999
y=x*100
print(abs(y))
#result='{:,.2f}%'.format(x*100)
print(round(y))
print(result)
print(int(y))


In [ ]:
import pandas as pd
import mysql.connector as sql
db=sql.connect(host='localhost',user='root',password='kobalan',database='phonepe')
cursor=db.cursor()
year=2018
quarter=2
cursor.execute(f'select * from agg_Transaction')
df_Trans1=pd.DataFrame(cursor.fetchall(),columns=cursor.column_names)
df_Trans2=df_Trans1[(df_Trans1['Year']==int(year)) & (df_Trans1['Quarter']==int(quarter))]
Transaction=df_Trans2['Transaction_count'].sum()
Payment=df_Trans2['Transaction_amount'].sum()
Average=Payment//Transaction
# df_Trans2= df_Trans2.groupby(["State"])[["Transaction_count", "Percentage"]].sum().sort_values(by='Transaction_count',ascending=False)
# df_Trans2.reset_index(inplace= True)
# df_Trans2['State'] = df_Trans2['State'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Daman & Diu")  
#df_Trans2['Percentage']=df_Trans2['Percentage'].apply(lambda x: x*100).apply(lambda x: '{:,.0f}%'.format(int(x)))
# df_Trans2['All_Transaction'] = df_Trans2['Transaction_count'].apply(lambda x: round(x)).apply(lambda x: comma_Seperator(x))
print(Transaction)
print(Average)
print(Payment)

In [ ]:
year=2018
quarter=2
import plotly_express as plt
result="Tamil Nadu"
cursor.execute(f' select * from top_transaction where State="Tamil Nadu" ')
#cursor.execute(f'select State,Pincode,sum(Transaction_amount) as Total_Payment_value,sum(Transaction_count) as Total_Transaction  from top_transaction where year={year} and quarter={quarter} and State={result} group by Pincode,State order by Total_Payment_value desc')
df_Trans5=pd.DataFrame(cursor.fetchall(),columns=cursor.column_names)
# df_Trans6=df_Trans5[df_Trans5['State']==result]
# pincode1=df_Trans6.Pincode.tolist()
# pincode2=[str(i) for i in pincode1]
# count3=df_Trans6.Total_Payment_value.tolist()
# fig3=go.Figure(data=[go.Bar(
#             x=pincode2,
#             y=count3,                                                   
#             name="Pincode",                                    
#             marker=dict(color='blue'),   
#             orientation="v"),
# ],
# layout=go.Layout(
#     xaxis=dict(title="Pincode"),
#     yaxis=dict(title="TRANSACTIONS"),
#     font=dict(
#         family="Neutro",
#         size=30,
#         color="RebeccaPurple",
#         variant="small-caps",
# )                                          
# ))
# fig3.update_layout(barmode="stack")
# fig3.update_layout(
#     width=800,
#     height=600,)

print(df_Trans5.head())


In [ ]:
import plotly.graph_objects as go
cursor.execute(f'select State,District,sum(Transaction_amount) as Total_Payment_value,sum(Transaction_count) as Total_Transaction  from map_transaction where year={year} and quarter={quarter} group by district,State order by Total_Payment_value desc')
df_Trans3=pd.DataFrame(cursor.fetchall(),columns=cursor.column_names)
df_Trans4=df_Trans3[df_Trans3['State']==result]
district=df_Trans4.District.tolist()
count2=df_Trans4.Total_Transaction.tolist()
fig2=go.Figure(data=[go.Bar(
            x=district,
            y=count2,                                                   
            name="District",                                    
            marker=dict(color='blue'),   
            orientation="v"),
],
layout=go.Layout(
    xaxis=dict(title="DISTRICT"),
    yaxis=dict(title="TRANSACTIONS"),
    font=dict(
        family="Neutro",
        size=30,
        color="RebeccaPurple",
        variant="small-caps",
)                                          
))
fig2.update_layout(barmode="stack")
fig2.update_layout(
    width=800,
    height=600,)


In [ ]:
count2

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import mysql.connector as sql

# test = {'Name': ['Company A','Company B','Company C','Company D','Company E'], 'count': [11,40,18,32,5], 'color':['red','blue','green','red','green'], 'Access':['Closed','Half','Free','Closed','Free']}
# df = pd.DataFrame(data=test)
db=sql.connect(host='localhost',username='root',password='kobalan',database='phonepe',auth_plugin="mysql_native_password")
cursor=db.cursor()
cursor.execute(f'select brand as Brand, sum(Transaction_count) as Total_count from agg_user where year>=2018 group by brand order by brand')
df=pd.DataFrame(cursor.fetchall(),columns=cursor.column_names)
brand=df['Brand'].tolist()
Total_Value= df['Total_count'].tolist()
fig3=go.Figure(data=[go.Bar(
            x=brand,                                                           
            y=Total_Value,                                                 
            name="Bfand",                                        
            marker=dict(color='brown'),   
            orientation="v"),
                                ],
    layout=go.Layout(
    xaxis=dict(title="Brand"),
    yaxis=dict(title="TRANSACTIONS"),
    font=dict(
        family="Neutro",
        size=15,
        color="RebeccaPurple",
        variant="small-caps",
)              
))
fig3.update_layout(
    width=600,
    height=400,

)

